In [ ]:
import os
import sys
from pathlib import Path

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.smr_datamodule import SMR_Data
from src.utils.device import print_data_info
import numpy as np

In [ ]:
data_dir = Path("D:\\SMR\\")
task_name = "2D"
subject_sessions_dict = {"S5": "all"}
loading_data_mode = "within_subject"
ival = "2s:11s:1ms"
bands = [8, 20]
chans = "*"
fallback_neighbors = 4
transform = None
normalize = {"norm_type": "std", "norm_axis": 0}
# normalize = None
process_noisy_channels = True
ignore_noisy_sessions = False

trial_type = "forced"

smr_datamodule = SMR_Data(data_dir=data_dir,
                          task_name=task_name,
                          trial_type=trial_type,
                          subject_sessions_dict=subject_sessions_dict,
                          loading_data_mode=loading_data_mode,
                          ival=ival,
                          bands=bands,
                          chans=chans,
                          fallback_neighbors=fallback_neighbors,
                          transform=transform,
                          normalize=normalize,
                          process_noisy_channels=process_noisy_channels,
                          ignore_noisy_sessions=ignore_noisy_sessions)

In [ ]:
smr_datamodule.prepare_dataloader_baseline()

In [ ]:
train_data = smr_datamodule.train_data
print_data_info(train_data)

In [ ]:
np.unique(train_data.y, return_counts=True)

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=False)
# enumerate the splits and summarize the distributions
for train_ix, test_ix in kfold.split(train_data, train_data.y):
 # select rows
 train_X, test_X = train_data[train_ix], train_data[test_ix]
 train_y, test_y = train_data.y[train_ix], train_data.y[test_ix]
 # summarize train and test composition
 train_0, train_1, train_2, train_3 = len(train_y[train_y==0]), len(train_y[train_y==1]), len(train_y[train_y==2]), len(train_y[train_y==3])
 test_0, test_1, test_2, test_3 = len(test_y[test_y==0]), len(test_y[test_y==1]), len(test_y[test_y==2]), len(test_y[test_y==3])
 print('>Train: 0=%d, 1=%d,2=%d, 3=%d, Test: 0=%d, 1=%d, 2=%d, 3=%d' % (train_0, train_1,train_2, train_3, test_0, test_1,test_2, test_3))

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_data.y), y=train_data.y)
class_weights_dict = dict(enumerate(class_weights))
class_weights_dict

In [ ]:
run_path = "../results_csv/2D_results_filters/run_3/"
os.makedirs(run_path, exist_ok=True)

### Cross Validation 

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=False)

## BBCpy toolbox

### CSP MNE

In [ ]:
import pandas as pd
from bbcpy.pipeline import make_pipeline
import bbcpy.functions.helpers as helpers
from bbcpy.functions.base import ImportFunc
from mne.decoding import CSP
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score


var = ImportFunc(np.var, axis=2)
# MNE csp 
n_components=[4 ,6, 8, 10]
reg=0.2 #float | str | None (default None)allow regularization for covariance estimation|If float (between 0 and 1), shrinkage is used/
log=False  # None | bool (default None)
cov_est='concat' #‘concat’ | ‘epoch’ (default ‘concat’)
transform_into='average_power' #‘average_power’ : the average power of each spatial filter| ‘csp_space’ :  the data in CSP space(default ‘average_power’)
norm_trace=True # bool (default False) Normalize class covariance by its trace.
cov_method_params=None #dict | None Parameters to pass to mne.compute_covariance().
rank=None #None | ‘info’ | ‘full’ | dict
component_order='mutual_info' #‘mutual_info’ | ‘alternate’ (default ‘mutual_info’) 

results = []
for filter in n_components:
    csp = CSP(n_components=filter,
              reg=reg,
              log=log,
              cov_est=cov_est,
              transform_into=transform_into,
              norm_trace=norm_trace,
              cov_method_params=cov_method_params,
              rank=rank,
              component_order=component_order)
    
    csp_pipeline = make_pipeline(   csp,
                                    var,
                                    np.log,
                                    LDA()
                                )
    
    # Perform cross-validation and store the results
    scores = cross_val_score(csp_pipeline, train_data, train_data.y, cv=cv, error_score='raise')

    # Store results for each fold
    for i, score in enumerate(scores):
        results.append([filter, i+1, score])

In [ ]:
# Create a DataFrame from the results
file_name = 'filter_results_csp_MNE.csv'
results_df = pd.DataFrame(results, columns = ['filter', 'fold', 'score'])
results_df.to_csv(os.path.join(run_path, file_name), index=False)

# Calculate mean and standard deviation for each excllev value
summary_df = results_df.groupby('filter')[['score']].agg(['mean', 'std'])
summary_df
file_name = 'filter_summary_csp_MNE.csv'
summary_df.to_csv(os.path.join(run_path, file_name))

## Riemann 

In [ ]:
import pyriemann
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import bbcpy
from bbcpy.functions.artireject import AverageVariance
from bbcpy.pipeline import make_pipeline
from bbcpy.functions.base import ImportFunc

In [ ]:
# Define the values of excllev to investigate
excllev_values = [None, 1.2, 1.8, 2, 4, 8, 10, 20, 100]
estimator = 'lwf'
# Initialize an empty list to store results
results = []

for excllev in excllev_values:
    # AverageVariance(excllev=filter, estimator=estimator)
    Cov = ImportFunc(bbcpy.functions.statistics.cov, estimator=estimator)
    riemann_pipeline = make_pipeline(AverageVariance(excllev=excllev, estimator=estimator),
                                     Cov,
                                     pyriemann.classification.SVC(metric='riemann', class_weight=class_weights_dict, decision_function_shape="ovo"))
    # Perform cross-validation and store the results
    scores = cross_val_score(riemann_pipeline, train_data, train_data.y, cv=cv, error_score='raise')
   
    # Store results for each fold
    for i, score in enumerate(scores):
        results.append([excllev, i+1, score])

In [ ]:
# Create a DataFrame from the results
results_df = pd.DataFrame(results, columns=['excllev', 'fold', 'score'])
file_name = 'excllev_results_Riemann_bbcpy.csv'
results_df.to_csv(os.path.join(run_path, file_name), index=False)

# Calculate mean and standard deviation for each excllev value
summary_df = results_df.groupby('excllev')[['score']].agg(['mean', 'std'])
summary_df
file_name = 'excllev_summary_Riemann_bbcpy.csv'
summary_df.to_csv(os.path.join(run_path, file_name))

In [ ]:
summary_df

## Tangent Rieamnn

In [ ]:
# Define the values of excllev to investigate
excllev_values = [None, 1.2, 1.8, 2, 4, 8, 10, 20, 100]
estimator = 'lwf'
# Initialize an empty list to store results
results = []
for excllev in excllev_values:

    Triemann_pipeline = make_pipeline(AverageVariance(excllev=excllev, estimator=estimator),
                                      ImportFunc(bbcpy.functions.statistics.cov, estimator=estimator),
                                      pyriemann.tangentspace.TangentSpace(),
                                      sklearn.svm.SVC(class_weight=class_weights_dict, decision_function_shape="ovo"))
    # Perform cross-validation and store the results
    scores = cross_val_score(Triemann_pipeline, train_data, train_data.y, cv=cv, error_score='raise')
    
    # Store results for each fold
    for i, score in enumerate(scores):
        results.append([excllev, i+1, score])

In [ ]:
# Create a DataFrame from the results
results_df = pd.DataFrame(results, columns=['excllev', 'fold', 'score'])
file_name = 'excllev_results_TRiemann_bbcpy.csv'
results_df.to_csv(os.path.join(run_path, file_name), index=False)

# Calculate mean and standard deviation for each excllev value
summary_df = results_df.groupby('excllev')[['score']].agg(['mean', 'std'])
summary_df
file_name = 'excllev_summary_TRiemann_bbcpy.csv'
summary_df.to_csv(os.path.join(run_path, file_name))

In [ ]:
summary_df